# Twitter Pipeline v2

## Imports

In [1]:
path = '../'
import pandas as pd
import re
#import custom functions
import sys
from PYTHON_FILES.LogReg_Searches import LogRegSearches

## Variables

In [2]:
# descriptions training set -> v2 = musow+mji descriptions vs summarized scrapes from twitter searches  
archive_desc_training_v2 = pd.read_pickle(path+'LOGREG_RELEVANCE/TRAINING_SETS/archive_desc_training_v2.pkl')

# twitter training set -> v1 = tweets from bigrams vs tweets for digital humanities and music company 
twitter_training_set_v1 = pd.read_pickle(path+'LOGREG_RELEVANCE/TRAINING_SETS/twitter_training_v1.pkl')

## Training twitter and descriptions classifiers

This is a ONE TIME operation. The models are pickled and loaded later to predict new results from LOGREG_RELEVANCE/MODELS folder. 

In [ ]:
# one time training on twitter
#twitter_training_model = LogRegSearches.train(twitter_training_set_v1, 'tweet', 'Target', 10, 'precision', 'twitter', path)

# one time training on resources
#resource_training_model = LogRegSearches.train(archive_desc_training_v2, 'Description', 'Target', 10, 'f1','resources',path)

## Query Twitter

Calls Twitter API with a list of keywords and return results as raw csv and clean pickle in TWITTER_SEARCHES/RAW_SEARCHES folder.

In [ ]:
#load token
token = 'AAAAAAAAAAAAAAAAAAAAAJgsNAEAAAAAQcsgbUnOJJmqmU483%2F8x6n9V1i8%3Df0qaEo9cV1sWP4eyNQ6E9s8BiRjvFTSN9mSqithe8uIXSNP68x'
#a selection of keywords based on MJI and musoW datasets

#choose KW list 
##best performing list based on v1 tests
better_keywords = ['audio file', 'music archive', 'music collection', 'music library', 'sheet music', 'sound archive', 'sound recording']

#custom timeframe for searching
start = ['2022-05-01T00:00:00.000Z', '2022-05-02T00:00:00.000Z', '2022-05-03T00:00:00.000Z', '2022-05-04T00:00:00.000Z', '2022-05-05T00:00:00.000Z', '2022-05-06T00:00:00.000Z', '2022-05-07T00:00:00.000Z']
end = ['2022-05-01T23:59:59.000Z', '2022-05-02T23:59:59.000Z', '2022-05-03T23:59:59.000Z', '2022-05-04T23:59:59.000Z', '2022-05-05T23:59:59.000Z', '2022-05-06T23:59:59.000Z', '2022-05-07T23:59:59.000Z']

#choose search option 
## search last week
LogRegSearches.search_twitter_weekly(token, better_keywords, 500, 500)
## search custom timeframe
LogRegSearches.search_twitter_custom(token, better_keywords, start, end, 500, 500)

## Classify tweets

In [3]:
#load all search results into a single dataframe 
tweets_to_classify = LogRegSearches.tweets_to_classify(path, '2022-05-01_5-05.pkl')

Total tweets to classify: 12333


In [4]:
#run classification and get links from results
predicted_tweets, twitter_link_list = LogRegSearches.predict_twitter(path, 'twitter', tweets_to_classify, 'tweet', 1)

Total tweets classified: 1225


## Scrape URLS

In [ ]:
#scrape URL list and return a DF for resource classification
scraped_links = LogRegSearches.scrape_links(twitter_link_list, predicted_tweets, 'all_keywords_2022-05-01_5-05_scrapes')

## Classify web resources

In [6]:
predicted_resources = LogRegSearches.predict_resource(path, 'resources_v2', scraped_links, 'Description', 1, 'all_keywords_2022-05-01_5-05')

,tweet,Prediction,Score,Probability,Input Length,URL,Search KW,Title,Description
0,"""The Silent Film Sound and Music Archive (http...",1,4.390663,0.987759,229,http://www.sfsma.org,"""music archive"" -is:retweet",Silent Film Sound & Music Archive – a digital ...,"Since 2014, we've been digitizing and sharing..."
1,Listen to the most recent episode of my podcas...,1,3.697857,0.975822,169,https://anchor.fm/educa-web-radio/episodes/Mus...,"""music history"" -is:retweet",Music History com Fulvio Cristofoli - The Mote...,"Music History, desenvolvido e apresentado por..."
2,Check out this sheet music video I made of Laz...,1,3.498099,0.970634,127,https://ift.tt/pDMjZQh,"""sheet music"" -is:retweet",Reddit - Alla scoperta di ciò che ti piace,Check out this sheet music video I made of La...
3,BBF releases the May 2022 edition of FORUM VIE...,1,3.467769,0.969757,65,http://brokersforumofindia.com/magazines.php,"""digital edition"" -is:retweet",http://brokersforumofindia.com/magazines.php,An Award Winning - Monthly Capital Markets and...
4,"Happy birthday ! The Beaver, LSE Students' Uni...",1,3.453722,0.969342,293,https://digital.library.lse.ac.uk/browse#beaver,"""digital library"" -is:retweet",Browse | LSE Digital Library,"A collection of newspapers, journals, pamphle..."
...,...,...,...,...,...,...,...,...,...
255,Looking for troubles with Steve Erection Arous...,1,0.033814,0.508453,238,https://gruenholtz.com/product/fine-art-of-aro...,"""digital collection"" -is:retweet",Fine Art Of Arousal - Digital Collection - Gru...,The Fine Art of Erections is the very first p...
256,A fellow Brit wrote this in the Telegraph 30 ...,1,0.024075,0.506018,476,https://bit.ly/3rZrdqv,"""music history"" -is:retweet",History of music in the American Midwest,The American Midwest has often been sentiment...
257,Voyager by Hunter Gregory| J.W. Pepper Sheet M...,1,0.021628,0.505407,195,https://www.jwpepper.com/Voyager/11393487.item...,"""sheet music"" -is:retweet",Voyager by Hunter Gregory| J.W. Pepper Sheet M...,The piece uses carefully designed eighth-note...
258,Current status of ManiRatnam's PonniyinSelvan ...,1,0.017407,0.504352,187,https://m.behindwoods.com/tamil-movies-cinema-...,"""sound recording"" -is:retweet",Mani Ratnam and AR Rahman's Ponniyin Selvan Mo...,Director Mani Ratnam is directing the two-par...
